In [1]:
import json
import numpy as np
import skimage
import tifffile
import os
import shutil
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cv2
from PIL import Image

In [2]:
from charminal import *

print(f'EMOJI_BEGIN: {EMOJI_BEGIN}')
print(f'EMOJI_ERROR: {EMOJI_ERROR}')
print(f'EMOJI_WARNING: {EMOJI_WARNING}')
print(f'EMOJI_HINT: {EMOJI_HINT}')
print(f'EMOJI_SAVING: {EMOJI_SAVING}')
print(f'EMOJI_TIME: {EMOJI_TIME}')
print(f'EMOJI_DEBUG: {EMOJI_DEBUG}')

EMOJI_BEGIN: 🚀
EMOJI_ERROR: 💥
EMOJI_HINT: 💡
EMOJI_SAVING: 💾
EMOJI_TIME: ⌛
EMOJI_DEBUG: 🔔


In [22]:
# Load the JSON file
base_dir = '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train'

# Read the COCO JSON segmentation file from the dataset
with open(f'{base_dir}/_annotations.coco.json', 'r') as file:
  data = json.load(file)

In [23]:
# Print the first 10 image filenames
for img in data['images'][:10]:
  print(img['file_name'])

01605_jpeg.rf.0399606de41dae32925e6662b85ef684.jpg
G0026625_JPG.rf.0984d2648ddd2caf60254ea473825612.jpg
0000085_PNG.rf.0095b22d92a65eb9ee80399fe24e4b7b.jpg
G0026919_JPG.rf.0aa2895eeac462936d899e569c74215e.jpg
G0026617_JPG.rf.03673452ea5193f6686b992b370fa85b.jpg
G0026580_JPG.rf.1750af11c3c23ad1e7ea3c1f1a7cb171.jpg
0000183_JPG.rf.39f998130a5ffbc88a827965f7b838b0.jpg
01532_jpeg.rf.a235c82ebb9bd70f796accbac04f0fe3.jpg
0000544_PNG.rf.341ac2bfb80349af6853cd1634ee206d.jpg
01777_jpeg.rf.85b7d77af7accba71428e17182797479.jpg


In [6]:
# Print the first few filenames
for category in data['categories'][:10]:
  print(f"{category['id']}: {category['name']}")

0: Aerial-Images
1: building
2: land
3: road
4: vegetated
5: water


# Understanding The Structure of COCO JSON

In [69]:
# Function to print the structure of the COCO JSON file
def printStructure(d, indent=0):
  spacing = '  '
  # Print the structure of a dictionary list

  # If the input is a dictionary
  if isinstance (d, dict):
    for key, value in d.items():
      print('  '*indent + str(key))
      printStructure(value, indent+1)

  # If the input is a list
  if isinstance(d, list):
    # print('  ' * indent + 'List of length {} containing:]'.format(len(d)))
    print(f'{spacing * indent}List of length {len(d)} containing:')
    if d:
      printStructure(d[0], indent+1)

In [70]:
printStructure(data)

info
  year
  version
  description
  contributor
  url
  date_created
licenses
  List of length 1 containing:]
    id
    url
    name
categories
  List of length 6 containing:]
    id
    name
    supercategory
images
  List of length 47 containing:]
    id
    license
    file_name
    height
    width
    date_captured
annotations
  List of length 1121 containing:]
    id
    image_id
    category_id
    bbox
      List of length 4 containing:]
    area
    segmentation
      List of length 1 containing:]
        List of length 162 containing:]
    iscrowd


In [8]:
# Colors for each classes
Building_distinct = np.asarray([255, 0, 0])
Land_distinct = np.asarray([255, 255, 0])
Road_distinct = np.asarray([0, 0, 255])
Vegetation_distinct = np.asarray([0, 255, 0])
Unlabeled_distinct = np.asarray([255, 0, 255])

In [17]:
from PIL import ImageColor

def decideColor(category_id):
  """
    1: building
    2: land
    3: road
    4: vegetated
    5: water
  """
  Building    = np.asarray(ImageColor.getcolor('#E09950', 'RGB'), dtype=np.uint8) # [224 153  80]
  Land        = np.asarray(ImageColor.getcolor('#F7ECCF', 'RGB'), dtype=np.uint8) # [247 236 207]
  Road        = np.asarray(ImageColor.getcolor('#B5C3D1', 'RGB'), dtype=np.uint8) # [181 195 209]
  Vegetation  = np.asarray(ImageColor.getcolor('#6FA96F', 'RGB'), dtype=np.uint8) # [111 169 111] 
  Water       = np.asarray(ImageColor.getcolor('#90DAEE', 'RGB'), dtype=np.uint8) # [144 218 238]
  Unlabeled   = np.asarray(ImageColor.getcolor('#C9AF9A', 'RGB'), dtype=np.uint8) # [253 249 239]

  if category_id == 1:
    return tuple(Building)
    # return tuple(Building_distinct)
    # return list(Building / 255.0)
  elif category_id == 2:
    return tuple(Land)
    # return tuple(Land_distinct)
    # return list(Land / 255.0)
  elif category_id == 3:
    return tuple(Road)
    # return tuple(Road_distinct)
    # return list(Road / 255.0)
  elif category_id == 4:
    return tuple(Vegetation)
    # return tuple(Vegetation_distinct)
    # return list(Vegetation / 255.0)
  else:
    print('[\U0001F6A8] Outsider detected')
    return tuple(Unlabeled)
    # return tuple(Unlabeled_distinct)

In [139]:
from PIL import ImageColor

def decideColorGPT(category_id):
  """
    1: building
    2: land
    3: road
    4: vegetated
    5: water
  """
  Building    = np.asarray(ImageColor.getcolor('#E09950', 'RGB')) # [224 153  80]
  Land        = np.asarray(ImageColor.getcolor('#F7ECCF', 'RGB')) # [247 236 207]
  Road        = np.asarray(ImageColor.getcolor('#B5C3D1', 'RGB')) # [181 195 209]
  Vegetation  = np.asarray(ImageColor.getcolor('#6FA96F', 'RGB')) # [111 169 111] 
  Water       = np.asarray(ImageColor.getcolor('#90DAEE', 'RGB')) # [144 218 238]
  # Unlabeled   = np.asarray(ImageColor.getcolor('#FDF9EF', 'RGB')) # [253 249 239]
  Unlabeled   = np.asarray(ImageColor.getcolor('#C9AF9A', 'RGB')) # [253 249 239]
  
  colors = {
    1: tuple(Building),
    2: tuple(Land),
    3: tuple(Road),
    4: tuple(Vegetation),
    5: tuple(Water)
  }

  return colors.get(category_id, Unlabeled)

In [72]:
print(decideColor(1))

[0.8784313725490196, 0.6, 0.3137254901960784]


In [10]:
Building    = np.asarray(ImageColor.getcolor('#E09950', 'RGB')) # [224 153  80]
Land        = np.asarray(ImageColor.getcolor('#F7ECCF', 'RGB')) # [247 236 207]
Road        = np.asarray(ImageColor.getcolor('#B5C3D1', 'RGB')) # [181 195 209]
Vegetation  = np.asarray(ImageColor.getcolor('#6FA96F', 'RGB')) # [111 169 111]
Water       = np.asarray(ImageColor.getcolor('#90DAEE', 'RGB')) # [144 218 238]
# Unlabeled   = np.asarray(ImageColor.getcolor('#FDF9EF', 'RGB'))[::-1] # [253 249 239]
Unlabeled   = np.asarray(ImageColor.getcolor('#C9AF9A', 'RGB')) # [253 249 239]
print(tuple(Building))
print(tuple(Land))
print(tuple(Road))
print(tuple(Vegetation))
print(tuple(Water))
print(tuple(Unlabeled))

(224, 153, 80)
(247, 236, 207)
(181, 195, 209)
(111, 169, 111)
(144, 218, 238)
(201, 175, 154)


In [11]:
colors = {
  1: tuple(np.asarray(ImageColor.getcolor('#E09950', 'RGB'))),  # Building
  2: tuple(np.asarray(ImageColor.getcolor('#F7ECCF', 'RGB'))),  # Land
  3: tuple(np.asarray(ImageColor.getcolor('#B5C3D1', 'RGB'))),  # Road
  4: tuple(np.asarray(ImageColor.getcolor('#6FA96F', 'RGB'))),  # Vegetation
  5: tuple(np.asarray(ImageColor.getcolor('#90DAEE', 'RGB'))),  # Water
  6: tuple(np.asarray(ImageColor.getcolor('#FDF9EF', 'RGB'))),  # Unlabeled
}

# colors = {
  # 1: tuple(Building_distinct),  # Building
  # 2: tuple(Land_distinct),  # Land
  # 3: tuple(Road_distinct),  # Road
  # 4: tuple(Vegetation_distinct),  # Vegetation
  # 5: tuple(Unlabeled_distinct)  # Unlabeled
# }

def imagesWithAnnotations(image_paths, annotations, colors, output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  fig, axs = plt.subplots(5, 2, figsize=(30, 30))
  
  for ax, img_path in zip(axs.ravel(), image_paths):
    unlabeled_color = tuple(Unlabeled)

    # Load image using OpenCV and convert it from BGR to RGB color space
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    canvas = np.full(image.shape, unlabeled_color, dtype=np.uint8)
    
    ax.imshow(image)
    ax.axis('off')  # Turn off the axes

    # Get image filename to match with annotations
    img_filename = os.path.basename(img_path)
    img_id = next(item for item in annotations['images'] if item["file_name"] == img_filename)['id']
        
    # Filter annotations for the current image
    img_annotations = [ann for ann in annotations['annotations'] if ann['image_id'] == img_id]
        
    for ann in img_annotations:
      category_id = ann['category_id']
      # color = decideColorGPT(category_id=category_id)
      color = decideColor(category_id=category_id)

      # Display segmentation polygon
      for seg in ann['segmentation']:

        poly = np.array(seg).reshape((-1, 2))
        cv2.fillPoly(canvas, [np.int32(poly)], color=(int(color[0]), int(color[1]), int(color[2])))
    
    ax.imshow(canvas)

  plt.tight_layout()
  plt.show()

In [27]:
# RUN THIS (20240812)
def imagesWithAnnotations(image_paths, annotations, out_image_dir, out_mask_dir):
  if not os.path.exists(out_image_dir):
    print(f"[{EMOJI_WARNING}] {out_image_dir} doesn't exist. Creating...")
    os.makedirs(out_image_dir)

  if not os.path.exists(out_mask_dir):
    print(f"[{EMOJI_WARNING}] {out_mask_dir} doesn't exist. Creating...")
    os.makedirs(out_mask_dir)

  # fig, axs = plt.subplots(5, 2, figsize=(30, 30))

  for i, img_path in enumerate(image_paths):
    # unlabeled_color = Unlabeled_distinct
    unlabeled_color = Unlabeled

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    canvas = np.full(image.shape, unlabeled_color, dtype=np.uint8)

    img_filename = os.path.basename(img_path)
    img_id = next(item for item in annotations['images'] if item["file_name"] == img_filename)['id']
    
    img_annotations = [ann for ann in annotations['annotations'] if ann['image_id'] == img_id]

    for ann in img_annotations:
      category_id = ann['category_id']
      color = decideColor(category_id=category_id)
            
      # Display segmentation polygon
      for seg in ann['segmentation']:
        poly = np.array(seg).reshape((-1, 2))
        cv2.fillPoly(canvas, [np.int32(poly)], color=(int(color[0]), int(color[1]), int(color[2])))

    # Save the mask image
    image = Image.fromarray(image)
    canvas = Image.fromarray(canvas)

    # mask_filename = f'{img_filename}'
    mask_filename = f'{(i+20):04d}.png'
    # mask_filename = f'{mask_filename}.'
    # mask_filename = f'{img_filename[:-4]}.png'
    print(f'mask_filename: {mask_filename}')
    # print(f'{mask_filename[:-4]}.png')
    out_image_path = os.path.join(out_image_dir, mask_filename)
    out_mask_path = os.path.join(out_mask_dir, mask_filename)
    # cv2.imwrite(out_image_path, image)
    # cv2.imwrite(out_mask_path, canvas)
    image.save(out_image_path)
    canvas.save(out_mask_path)
    print(f'[{emoji_saving}] Saving image: {out_image_path}')
    print(f'[{emoji_saving}] Saving mask: {out_mask_path}')


In [28]:
# RUN THIS (CONT) (20240812)
all_image_files = [os.path.join(base_dir, img['file_name']) for img in data['images']]
print(all_image_files)
print(f'all_image_files len: {len(all_image_files)}')

dest_dir = '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train-png'

# out_image_dir = f'{base_dir}/images-png'
out_image_dir = f'{dest_dir}/images'
# out_mask_dir = f'{base_dir}/masks-png'
out_mask_dir = f'{dest_dir}/masks'
# random_image_files = random.sample(all_image_files, 10)
# display_type = 'seg'
# imagesWithAnnotations(all_image_files, data, colors, out_image_dir, out_mask_dir)
imagesWithAnnotations(all_image_files, data, out_image_dir, out_mask_dir)

['/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train/01605_jpeg.rf.0399606de41dae32925e6662b85ef684.jpg', '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train/G0026625_JPG.rf.0984d2648ddd2caf60254ea473825612.jpg', '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train/0000085_PNG.rf.0095b22d92a65eb9ee80399fe24e4b7b.jpg', '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train/G0026919_JPG.rf.0aa2895eeac462936d899e569c74215e.jpg', '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train/G0026617_JPG.rf.03673452ea5193f6686b992b370fa85b.jpg', '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train/G0026580_JPG.rf.1750af11c3c23ad1e7ea3c1f1a7cb171.jpg', '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train/0000183_JPG.rf.39f998130a5ffbc88a827965f7b838b0.jpg', '/home/pwnwas/Personal/Gamaforce/dataset/roboflow-plus-gading-coco/train/01532_jpeg.rf.a235c82ebb9bd70f796a